In [3]:
import numpy as np
import pandas as pd

def tasaCupon0(datos):
    r0 = []
    for per,dato in enumerate(datos):
        r = np.rate(per+1, 0, -dato[0]-sum([np.pv(r0[k], k+1, 0, dato[1]) for k in range(per)]), 1+dato[1])
        r0.append(r)
    return r0

def tasaFwd(Tasa0, all = True):
    fwd = []
    if all == True:
        for i in range(len(Tasa0)):
            r = ((1+Tasa0[i])**(i+1))/((1+Tasa0[i-1])**(i))-1
            fwd.append(float(format(r,'.4f')))
        return fwd

    elif all == False:
        i = int(input('Ingrese periodo inicial:'))
        f = int(input('Ingrese periodo final:'))
        r = (((1+Tasa0[f])**f)/((1+Tasa0[i])**i))**(f-i)**-1-1
        return format(r,'.2%')

def sensibilidad(vn, tmar, cupon, plazo, var_i):
    va = [-np.pv(tmar,i,0,cupon) for i in range(1,plazo+1)]
    va[-1] -= np.pv(tmar,plazo,0,1)
    x = va/sum(va)

    duracion = sum([(i+1)*j for i,j in enumerate(x)])
    convexidad = sum([((i+1)*(i+2)*j)/(1+tmar)**2 for i,j in enumerate(x)])
    dm = duracion/(1+tmar)
    eDuracion = var_i*-dm
    eConvexidad = 0.5*convexidad*var_i**2
    total = eDuracion+eConvexidad
    nuevoPrecio = np.pv(tmar,plazo,cupon*vn,vn)*(1+total)

    print(31*'=')
    print('{:20} {:10.2f}'.format('Duración', duracion))
    print('{:20} {:10.2f}'.format('Duración Modificada', dm))
    print('{:20} {:10.2f}'.format('Convexidad', convexidad))
    print('EFECTOS'.center(31,'-'))
    print('{:20} {:>10.2%}'.format('Duración', eDuracion))
    print('{:20} {:>10.2%}'.format('Convexidad', eConvexidad))
    print('{:20} {:>10.2%}'.format('Total', total))
    print(31*'-')
    print('{:20} {:>10.2f}'.format('Precio Nuevo', nuevoPrecio))
    print(31*'=')

def generarReporteBonos(Precios, Cupones):
    tupla = ((precio,cupon) for precio,cupon in zip(precios, cupones))
    Cupon0 = tasaCupon0(tupla)
    Forward = tasaFwd(Cupon0)

    print(35*'=')
    print('{:<5} {:10} {:10} {:10}'.format('t', 'TIR=CUPON', 'CUPON 0', 'FORWARD'))
    print(35*'=')
    for i in range(len(Cupones)):
        print('{:<5} {:^10.2%} {:^10.2%} {:^10.2%}'.format(i+1, Cupones[i], Cupon0[i], Forward[i]))
    print(35*'=')

In [4]:
vn = 100
tmar = 0.0289
cupon = 0.0
plazo = 4
var_i = 0.0055
sensibilidad(vn,tmar,cupon,plazo,var_i)


precios = [1 for i in range(8)]
cupones = [0.04,0.041,0.0430,0.0470,0.05,0.051,0.052,0.053]
generarReporteBonos(precios,cupones)

Duración                   4.00
Duración Modificada        3.89
Convexidad                18.89
------------EFECTOS------------
Duración                 -2.14%
Convexidad                0.03%
Total                    -2.11%
-------------------------------
Precio Nuevo             -87.35
t     TIR=CUPON  CUPON 0    FORWARD   
1       4.00%      4.00%      4.00%   
2       4.10%      4.10%      4.20%   
3       4.30%      4.31%      4.73%   
4       4.70%      4.74%      6.03%   
5       5.00%      5.06%      6.38%   
6       5.10%      5.17%      5.69%   
7       5.20%      5.28%      5.93%   
8       5.30%      5.39%      6.18%   


In [25]:
tupla = ((precio,cupon) for precio,cupon in zip(precios, cupones))
tasasCupones0 = tasaCupon0(tupla)
tasasForward = tasaFwd(tasasCupones0)
datos = list(zip(cupones,tasasCupones0, tasasForward))

pd.options.display.float_format = '{:,.2%}'.format

df = pd.DataFrame(datos,range(1,9),['TIR = CUPON', 'CUPON 0', 'FORWARD'])

df

,TIR = CUPON,CUPON 0,FORWARD
1,4.00%,4.00%,4.00%
2,4.10%,4.10%,4.20%
3,4.30%,4.31%,4.73%
4,4.70%,4.74%,6.03%
5,5.00%,5.06%,6.38%
6,5.10%,5.17%,5.69%
7,5.20%,5.28%,5.93%
8,5.30%,5.39%,6.18%


In [3]:
def tasaFwd(Tasa0, all = True):
    fwd = []
    if all == True:
        for i in range(len(Tasa0)):
            r = ((1+Tasa0[i])**(i+1))/((1+Tasa0[i-1])**(i))-1
            fwd.append(float(format(r,'.4f')))
        return fwd

    elif all == False:
        i = int(input('Ingrese periodo inicial:'))
        f = int(input('Ingrese periodo final:'))
        r = (((1+Tasa0[f])**f)/((1+Tasa0[i])**i))**(f-i)**-1-1
        return format(r,'.2%')
    
tasaFwd([0.04,0.041,0.0430,0.0470,0.05,0.051,0.052,0.053], False)

Ingrese periodo inicial:1
Ingrese periodo final:4


'5.30%'

In [27]:
def convertir_tasa(Tasa, PerTasa, PerCap):
    import numpy as np
    import pandas as pd
    per = 'Anual', 'Semestral', 'Quimestral', 'Cuatrimestral', 'Trimestral', 'Bimestral', 'Mensual', 'Bimensual', 'Semanal', 'Diaria'
    con = 1,2,12/5,3,4,6,12,24,52,360

    n = PerTasa
    m = PerCap

    if n == m:
        s = 'Tasa: {:.2%} efectiva {}'.format(Tasa,per[con.index(n)].lower())
    else:
        s = 'Tasa: {:.2%} {} comp. {}'.format(Tasa,per[con.index(n)].lower(), per[con.index(m)].lower())
    
    print('\n'+s)
    mat = []
    for i in range(len(per)):
        tasasNom = [con[i]/j*((1+Tasa*n/m)**(m/con[i])-1) for j in con]
        tasasCont = [np.log1p(Tasa*n/m)*m/j for j in con]
        efe = (1+Tasa*n/m)**(m/con[i])-1
        fila = mat.append([efe] + tasasNom)
    mat = np.array(mat)
    pd.options.display.float_format = '{:,.4%}'.format
    df = pd.DataFrame(mat,per,['Efect']+[i[:5] for i in per])
    return df

In [28]:

convertir_tasa(0.09,2,2)


Tasa: 9.00% efectiva semestral


,Efect,Anual,Semes,Quime,Cuatr,Trime,Bimes,Mensu,Bimen,Seman,Diari
Anual,18.8100%,18.8100%,9.4050%,7.8375%,6.2700%,4.7025%,3.1350%,1.5675%,0.7838%,0.3617%,0.0523%
Semestral,9.0000%,18.0000%,9.0000%,7.5000%,6.0000%,4.5000%,3.0000%,1.5000%,0.7500%,0.3462%,0.0500%
Quimestral,7.4456%,17.8695%,8.9348%,7.4456%,5.9565%,4.4674%,2.9783%,1.4891%,0.7446%,0.3436%,0.0496%
Cuatrimestral,5.9134%,17.7403%,8.8701%,7.3918%,5.9134%,4.4351%,2.9567%,1.4784%,0.7392%,0.3412%,0.0493%
Trimestral,4.4031%,17.6123%,8.8061%,7.3384%,5.8708%,4.4031%,2.9354%,1.4677%,0.7338%,0.3387%,0.0489%
Bimestral,2.9142%,17.4855%,8.7427%,7.2856%,5.8285%,4.3714%,2.9142%,1.4571%,0.7286%,0.3363%,0.0486%
Mensual,1.4467%,17.3599%,8.6800%,7.2333%,5.7866%,4.3400%,2.8933%,1.4467%,0.7233%,0.3338%,0.0482%
Bimensual,0.7207%,17.2976%,8.6488%,7.2073%,5.7659%,4.3244%,2.8829%,1.4415%,0.7207%,0.3326%,0.0480%
Semanal,0.3320%,17.2641%,8.6321%,7.1934%,5.7547%,4.3160%,2.8774%,1.4387%,0.7193%,0.3320%,0.0480%
Diaria,0.0479%,17.2397%,8.6198%,7.1832%,5.7466%,4.3099%,2.8733%,1.4366%,0.7183%,0.3315%,0.0479%


# Cuadro de tasas